In [9]:
from minitorch.engine import Value

from minitorch.nn import *

from minitorch.tensor import Tensor

from typing import List

# Use case: Creation of a Neural Network

In [2]:
class MLP(Module):
    def __init__(self):
        self.layers = Sequential(
            Linear(2,3),
            ReLU(),
            Linear(3,1),
            Sigmoid()
        )
    
    def forward(self, input : List[float]):
        
        # We convert the input to a tensor
        input = Tensor(input)

        # Now we do the forward pass
        return self.layers(input)

In [3]:
model = MLP()
res = model.forward([-10,10])
print(res)

Tensor([Value(data=0.4657842909200023, grad=0)])


In [4]:
def MSE(output : Tensor, label : Tensor):
    o = list(output)[0]
    l = list(label)[0]
    return (o - l)**2

In [5]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v.getChildren():
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n.getOperation():
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n.getOperation(), label = n.getOperation())
      # and connect this node to it
      dot.edge(uid + n.getOperation(), uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2.getOperation())

  return dot

In [6]:
data = [[0,0], [0,1], [1,0], [1,1]]
label = [[0,], [0,], [0,], [1,]] # AND function

lr = 0.5

epoch = 100
for i in range(epoch):
    for d, l in zip(data,label):
        
        output = model.forward(d)
        
        tensor_label = Tensor(l)

        model.zero_grad()
        loss = MSE(output, tensor_label)

        loss.backward()

        for p in model.parameters():
            p.data -= lr * p.grad
        

In [7]:
output = model.forward(data[3])
output

Tensor([Value(data=0.955935451977591, grad=0)])